# Image Only Input Models

In [1]:
import numpy as np
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import os
from skimage import io
import torch
from skimage import color
from torchvision import transforms
import torchvision.models as models
import ast
import time
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import cohen_kappa_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import pickle
from copy import deepcopy

In [2]:
if torch.cuda.is_available:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

## Transformations

In [3]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

## Dataset

In [4]:
class FundusDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with filename information.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        left_img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 3])
        right_img_name = os.path.join(self.root_dir, self.data_frame.iloc[idx, 4])

        left_image = io.imread(left_img_name)
        right_image = io.imread(right_img_name)

        if len(left_image.shape) > 2 and left_image.shape[2] == 4:
            left_image = left_image[:, :, 0]
        if len(right_image.shape) > 2 and right_image.shape[2] == 4:
            right_image = right_image[:, :, 0]

        if left_image.shape[-1] != 3:
            left_image = np.repeat(left_image[:, :, np.newaxis], 3, axis=2)
        if right_image.shape[-1] != 3:
            right_image = np.repeat(right_image[:, :, np.newaxis], 3, axis=2)

        image_class = self.data_frame.iloc[idx, -1]
        image_class = ast.literal_eval(image_class)
        image_class = torch.tensor(image_class, dtype=torch.int64)
        one_hot_image_class = F.one_hot(image_class, num_classes=2).squeeze(0)

        if self.transform:
            left_image = self.transform(left_image)
            right_image = self.transform(right_image)

        if left_image.dim() == 2:
            left_image = left_image.unsqueeze(0)
        if right_image.dim() == 2:
            right_image = right_image.unsqueeze(0)

        sample = {'left_image': left_image, 'right_image': right_image, 'class': one_hot_image_class}

        return sample


## Dataloading

In [5]:
# Set csv file paths
train_info_path = r'./OIA-ODIR/Training Set/Annotation/training_annotation_filtered.csv'
valid_info_path = r'./OIA-ODIR/Off-site Test Set/Annotation/validation_annotation_filtered.csv'
test_info_path = r'./OIA-ODIR/On-site Test Set/Annotation/testing_annotation_filtered.csv'

# Set root directory paths
train_root_dir = r'./OIA-ODIR/Training Set/Images'
valid_root_dir = r'./OIA-ODIR//Off-site Test Set/Images'
test_root_dir = r'./OIA-ODIR/On-site Test Set/Images'

# Transformed dataset for left and right fundus
transformed_dataset = {
    'train': FundusDataset(csv_file=train_info_path, root_dir=train_root_dir, transform=transform),
    'validate': FundusDataset(csv_file=valid_info_path, root_dir=valid_root_dir, transform=transform),
    'test': FundusDataset(csv_file=test_info_path, root_dir=test_root_dir, transform=transform)
}

## Model 1: Late Fusion (SUM)

In [6]:
class LateFusionSUM_Resnet18(nn.Module):
    def __init__(self):
        super(LateFusionSUM_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()  # Remove the fully connected layer
        self.fc = nn.Linear(num_features, 512)  # Fusion layer
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right):
        left_features = self.resnet18(left)
        right_features = self.resnet18(right)
        fused_features = left_features + right_features  # Element-wise sum fusion
        fused_features = self.fc(fused_features)
        outputs = [classifier(fused_features) for classifier in self.classifiers]
        # predictions = [torch.sigmoid(output) for output in outputs]  # Sigmoid activation for multi-label classification
        return torch.stack(outputs, dim=1)

## Model 2: Late Fusion (CONCAT)

In [7]:
class LateFusionCONCAT_Resnet18(nn.Module):
    def __init__(self):
        super(LateFusionCONCAT_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()  # Remove the fully connected layer
        self.fc = nn.Linear(num_features * 2, 512)  # Fusion layer
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right):
        left_features = self.resnet18(left)
        right_features = self.resnet18(right)
        fused_features = torch.cat((left_features, right_features), dim=1)  # concat features late fusion
        fused_features = self.fc(fused_features)
        outputs = [classifier(fused_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)

## Model 3: Late Fusion (PROD)

In [8]:
class LateFusionPROD_Resnet18(nn.Module):
    def __init__(self):
        super(LateFusionPROD_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()  # Remove the fully connected layer
        self.fc = nn.Linear(num_features, 512)  # Fusion layer
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right):
        left_features = self.resnet18(left)
        right_features = self.resnet18(right)
        fused_features = left_features * right_features  # Element-wise product fusion
        fused_features = self.fc(fused_features)
        outputs = [classifier(fused_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)

## Model 4: Early Fusion (SUM)

In [9]:
class EarlyFusionSUM_Resnet18(nn.Module):
    def __init__(self):
        super(EarlyFusionSUM_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()  # Remove the fully connected layer
        self.fc = nn.Linear(num_features, 512)  # Fusion layer
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right):
        summed_input = left + right
        fused_features = self.resnet18(summed_input)
        fused_features = self.fc(fused_features)
        outputs = [classifier(fused_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)


## Model 5: Early Fusion (CONCAT)

In [10]:
class EarlyFusionCONCAT_Resnet18(nn.Module):
    def __init__(self):
        super(EarlyFusionCONCAT_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        self.resnet18.conv1 = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False) # To accomodate concat early fusion
        num_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()
        self.fc = nn.Linear(num_features, 512)
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right):
        fused_input = torch.cat((left, right), dim=1)
        fused_features = self.resnet18(fused_input)
        fused_features = self.fc(fused_features)
        outputs = [classifier(fused_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)


## Model 6: Early Fusion (PROD)

In [11]:
class EarlyFusionPROD_Resnet18(nn.Module):
    def __init__(self):
        super(EarlyFusionPROD_Resnet18, self).__init__()
        self.resnet18 = models.resnet18(weights='DEFAULT')
        num_features = self.resnet18.fc.in_features
        self.resnet18.fc = nn.Identity()
        self.fc = nn.Linear(num_features, 512)
        self.classifiers = nn.ModuleList([nn.Linear(512, 2) for _ in range(8)])  # 8 classifiers for multi-label classification
        
    def forward(self, left, right):
        fused_input = left * right # Element-wise multiplication of left and right images
        fused_features = self.resnet18(fused_input)
        fused_features = self.fc(fused_features)
        outputs = [classifier(fused_features) for classifier in self.classifiers]
        return torch.stack(outputs, dim=1)

## Training Loop

In [12]:
torch.manual_seed(42)
np.random.seed(42)

def train_model(model, dataloader, optimizer, criterion, num_epochs=10, verbose=True, save_path=None):
    acc_dict = {'train':[],'validate':[],'test':[]}
    loss_dict = {'train':[],'validate':[],'test':[]}
    kappa_dict = {'validate': [],'test':[]}
    f1_dict = {'validate': [],'test':[]}
    auc_dict = {'validate': [],'test':[]}
    best_loss = float('inf')
    best_f1 = 0
    best_kappa = 0
    best_auc = 0
    phases = ['train','validate','test']
    since = time.time()
    for i in range(num_epochs):
        epoch_since = time.time()
        print('-'*10)
        print('Epoch: {}/{}'.format(i+1, num_epochs))
        print('-'*10)
        for p in phases:
            running_correct = 0
            running_loss = 0
            running_total = 0
            if p == 'train':
                model.train()
            else:
                model.eval()
            all_probabilities_pos = []
            all_predictions = []
            all_labels = []
            all_labels_pos = []

            for data in dataloader[p]:
                optimizer.zero_grad()
                left_images = data['left_image'].to(device)
                right_images = data['right_image'].to(device)
                labels = data['class'].to(device)
                outputs = model(left_images, right_images)
                num_classifiers = outputs.size(1)
                classifier_losses = []
                epoch_loss = 0 
                for j in range(num_classifiers):
                    classifier_outputs = outputs[:, j, :]
                    classifier_labels = labels[:, j, :].squeeze(dim=1)  # Squeeze extra dimension
                    classifier_loss = criterion(classifier_outputs, classifier_labels.float())
                    classifier_losses.append(classifier_loss)
                
                loss = sum(classifier_losses)
                epoch_loss += loss.item()
                
                probabilities = torch.sigmoid(outputs)
                probabilities_pos = probabilities[:, 1]
                
                labels_pos = labels[:, 1]
                
                predictions = (probabilities > 0.5).int()
                
                all_probabilities_pos.extend(probabilities_pos.cpu().detach().numpy())
                all_predictions.extend(predictions.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_labels_pos.extend(labels_pos.cpu().numpy())
                
                running_correct += torch.sum(predictions == labels).item()
                running_loss += loss.item()
                running_total += labels.size(0)
                if p== 'train':
                    loss.backward()
                    optimizer.step()
                
            epoch_acc = float(running_correct/running_total)
            epoch_loss = float(running_loss/running_total)
            
            
            if verbose or (i%10 == 0):
                print('Phase:{}, epoch loss: {:.4f}'.format(p, epoch_loss))
            acc_dict[p].append(epoch_acc)
            loss_dict[p].append(epoch_loss)
            
            if p == 'validate':
                all_labels_flat = np.concatenate(all_labels).flatten()
                all_predictions_flat = np.concatenate(all_predictions).flatten()
                all_labels_pos_flat = np.concatenate(all_labels_pos).flatten()
                all_probabilities_pos_flat = np.concatenate(all_probabilities_pos).flatten()
                
                kappa = cohen_kappa_score(all_labels_flat, all_predictions_flat)
                f1 = f1_score(all_labels_flat, all_predictions_flat, average='macro')
                auc = roc_auc_score(all_labels_pos, all_probabilities_pos) # macro
                
                print('Kappa: {:.4f} F1: {:.4f} AUC: {:.4f}'.format(kappa, f1, auc))
                
                kappa_dict[p].append(kappa)
                f1_dict[p].append(f1)
                auc_dict[p].append(auc)

                if f1 > best_f1:
                    best_f1 = f1
                    best_model_wts = deepcopy(model.state_dict())
                    best_epoch = i+1
                    best_kappa = kappa
                    best_auc = auc
                    
            if p == 'test':
                all_labels_flat = np.concatenate(all_labels).flatten()
                all_predictions_flat = np.concatenate(all_predictions).flatten()
                all_labels_pos_flat = np.concatenate(all_labels_pos).flatten()
                all_probabilities_pos_flat = np.concatenate(all_probabilities_pos).flatten()
                
                kappa = cohen_kappa_score(all_labels_flat, all_predictions_flat)
                f1 = f1_score(all_labels_flat, all_predictions_flat, average='macro')
                auc = roc_auc_score(all_labels_pos, all_probabilities_pos)
                
                print('Kappa: {:.4f} F1: {:.4f} AUC: {:.4f}'.format(kappa, f1, auc))
                
                kappa_dict[p].append(kappa)
                f1_dict[p].append(f1)
                auc_dict[p].append(auc)
                    
        if save_path:
            torch.save(best_model_wts, save_path.format(round(best_f1, 4)))
        epoch_time_elapsed = time.time() - epoch_since
        print('Epoch complete in {:.0f}m {:.0f}s'.format(epoch_time_elapsed // 60, epoch_time_elapsed % 60))
        
    time_elapsed = time.time() - since
    print('-'*10)
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best Val F1: {:4f} at Epoch {}'.format(best_f1, best_epoch))
    print('Corresponding Kappa: {:4f} and AUC: {:4f}'.format(best_kappa, best_auc))

    return acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict

## BS = 16, LR = 0.00001

In [13]:
bs = 16
lr = 0.00001

# Load data for left and right fundus
dataloader = {x: DataLoader(transformed_dataset[x], batch_size=bs, shuffle=True) for x in ['train', 'validate','test']}

In [14]:
lfs_model = LateFusionSUM_Resnet18()
lfs_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/LFS_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/LFS_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(lfs_model.parameters(), lr=lr)
lfs_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(lfs_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
lfs_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(lfs_results, file)

----------
Epoch: 1/10
----------
Phase:train, epoch loss: 0.1985
Phase:validate, epoch loss: 0.1634
Kappa: 0.7114 F1: 0.8557 AUC: 0.7101
Phase:test, epoch loss: 0.1643
Kappa: 0.7098 F1: 0.8549 AUC: 0.6679
Epoch complete in 12m 32s
----------
Epoch: 2/10
----------
Phase:train, epoch loss: 0.1525
Phase:validate, epoch loss: 0.1556
Kappa: 0.7278 F1: 0.8639 AUC: 0.7352
Phase:test, epoch loss: 0.1567
Kappa: 0.7177 F1: 0.8589 AUC: 0.6943
Epoch complete in 10m 15s
----------
Epoch: 3/10
----------
Phase:train, epoch loss: 0.1411
Phase:validate, epoch loss: 0.1519
Kappa: 0.7293 F1: 0.8646 AUC: 0.7584
Phase:test, epoch loss: 0.1542
Kappa: 0.7195 F1: 0.8598 AUC: 0.7206
Epoch complete in 9m 54s
----------
Epoch: 4/10
----------
Phase:train, epoch loss: 0.1321
Phase:validate, epoch loss: 0.1509
Kappa: 0.7369 F1: 0.8684 AUC: 0.7705
Phase:test, epoch loss: 0.1535
Kappa: 0.7240 F1: 0.8620 AUC: 0.7298
Epoch complete in 10m 14s
----------
Epoch: 5/10
----------
Phase:train, epoch loss: 0.1239
Phase:v

In [14]:
lfc_model = LateFusionCONCAT_Resnet18()
lfc_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/LFC_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/LFC_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(lfc_model.parameters(), lr=lr)
lfc_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(lfc_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
lfc_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(lfc_results, file)

----------
Epoch: 1/10
----------
Phase:train, epoch loss: 0.2025
Phase:validate, epoch loss: 0.1660
Kappa: 0.7068 F1: 0.8534 AUC: 0.6811
Phase:test, epoch loss: 0.1667
Kappa: 0.7138 F1: 0.8569 AUC: 0.6494
Epoch complete in 9m 26s
----------
Epoch: 2/10
----------
Phase:train, epoch loss: 0.1563
Phase:validate, epoch loss: 0.1555
Kappa: 0.7174 F1: 0.8587 AUC: 0.7217
Phase:test, epoch loss: 0.1566
Kappa: 0.7210 F1: 0.8605 AUC: 0.6861
Epoch complete in 9m 6s
----------
Epoch: 3/10
----------
Phase:train, epoch loss: 0.1446
Phase:validate, epoch loss: 0.1516
Kappa: 0.7225 F1: 0.8612 AUC: 0.7367
Phase:test, epoch loss: 0.1533
Kappa: 0.7206 F1: 0.8603 AUC: 0.6961
Epoch complete in 8m 57s
----------
Epoch: 4/10
----------
Phase:train, epoch loss: 0.1355
Phase:validate, epoch loss: 0.1502
Kappa: 0.7220 F1: 0.8610 AUC: 0.7573
Phase:test, epoch loss: 0.1523
Kappa: 0.7251 F1: 0.8625 AUC: 0.7027
Epoch complete in 8m 53s
----------
Epoch: 5/10
----------
Phase:train, epoch loss: 0.1257
Phase:valid

In [14]:
lfp_model = LateFusionPROD_Resnet18()
lfp_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/LFP_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/LFP_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(lfp_model.parameters(), lr=lr)
lfp_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(lfp_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
lfp_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(lfp_results, file)

----------
Epoch: 1/10
----------
Phase:train, epoch loss: 0.2253
Phase:validate, epoch loss: 0.1759
Kappa: 0.7053 F1: 0.8527 AUC: 0.6568
Phase:test, epoch loss: 0.1784
Kappa: 0.7015 F1: 0.8507 AUC: 0.6235
Epoch complete in 11m 30s
----------
Epoch: 2/10
----------
Phase:train, epoch loss: 0.1588
Phase:validate, epoch loss: 0.1630
Kappa: 0.7134 F1: 0.8567 AUC: 0.7007
Phase:test, epoch loss: 0.1665
Kappa: 0.7089 F1: 0.8545 AUC: 0.6677
Epoch complete in 9m 2s
----------
Epoch: 3/10
----------
Phase:train, epoch loss: 0.1445
Phase:validate, epoch loss: 0.1584
Kappa: 0.7146 F1: 0.8573 AUC: 0.7322
Phase:test, epoch loss: 0.1615
Kappa: 0.7126 F1: 0.8563 AUC: 0.6940
Epoch complete in 8m 59s
----------
Epoch: 4/10
----------
Phase:train, epoch loss: 0.1347
Phase:validate, epoch loss: 0.1590
Kappa: 0.7189 F1: 0.8595 AUC: 0.7398
Phase:test, epoch loss: 0.1632
Kappa: 0.7117 F1: 0.8558 AUC: 0.6976
Epoch complete in 8m 17s
----------
Epoch: 5/10
----------
Phase:train, epoch loss: 0.1264
Phase:vali

In [14]:
efs_model = EarlyFusionSUM_Resnet18()
efs_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/EFS_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/EFS_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(efs_model.parameters(), lr=lr)
efs_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(efs_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
efs_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(efs_results, file)

----------
Epoch: 1/10
----------
Phase:train, epoch loss: 0.2211
Phase:validate, epoch loss: 0.1740
Kappa: 0.7093 F1: 0.8547 AUC: 0.6696
Phase:test, epoch loss: 0.1745
Kappa: 0.7089 F1: 0.8545 AUC: 0.6260
Epoch complete in 9m 36s
----------
Epoch: 2/10
----------
Phase:train, epoch loss: 0.1614
Phase:validate, epoch loss: 0.1609
Kappa: 0.7152 F1: 0.8576 AUC: 0.7120
Phase:test, epoch loss: 0.1614
Kappa: 0.7131 F1: 0.8565 AUC: 0.6592
Epoch complete in 10m 1s
----------
Epoch: 3/10
----------
Phase:train, epoch loss: 0.1487
Phase:validate, epoch loss: 0.1578
Kappa: 0.7217 F1: 0.8609 AUC: 0.7311
Phase:test, epoch loss: 0.1585
Kappa: 0.7159 F1: 0.8579 AUC: 0.6807
Epoch complete in 9m 29s
----------
Epoch: 4/10
----------
Phase:train, epoch loss: 0.1401
Phase:validate, epoch loss: 0.1552
Kappa: 0.7217 F1: 0.8609 AUC: 0.7497
Phase:test, epoch loss: 0.1575
Kappa: 0.7119 F1: 0.8560 AUC: 0.6873
Epoch complete in 9m 25s
----------
Epoch: 5/10
----------
Phase:train, epoch loss: 0.1321
Phase:vali

In [14]:
efc_model = EarlyFusionCONCAT_Resnet18()
efc_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/EFC_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/EFC_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(efc_model.parameters(), lr=lr)
efc_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(efc_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
efc_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(efc_results, file)

----------
Epoch: 1/10
----------
Phase:train, epoch loss: 0.2235
Phase:validate, epoch loss: 0.1784
Kappa: 0.7121 F1: 0.8561 AUC: 0.6097
Phase:test, epoch loss: 0.1784
Kappa: 0.7082 F1: 0.8541 AUC: 0.6019
Epoch complete in 11m 5s
----------
Epoch: 2/10
----------
Phase:train, epoch loss: 0.1671
Phase:validate, epoch loss: 0.1691
Kappa: 0.7119 F1: 0.8559 AUC: 0.6657
Phase:test, epoch loss: 0.1688
Kappa: 0.7077 F1: 0.8538 AUC: 0.6345
Epoch complete in 9m 40s
----------
Epoch: 3/10
----------
Phase:train, epoch loss: 0.1576
Phase:validate, epoch loss: 0.1650
Kappa: 0.7111 F1: 0.8556 AUC: 0.6974
Phase:test, epoch loss: 0.1643
Kappa: 0.7088 F1: 0.8544 AUC: 0.6522
Epoch complete in 9m 49s
----------
Epoch: 4/10
----------
Phase:train, epoch loss: 0.1504
Phase:validate, epoch loss: 0.1623
Kappa: 0.7154 F1: 0.8577 AUC: 0.7160
Phase:test, epoch loss: 0.1632
Kappa: 0.7123 F1: 0.8562 AUC: 0.6603
Epoch complete in 9m 58s
----------
Epoch: 5/10
----------
Phase:train, epoch loss: 0.1442
Phase:vali

In [14]:
efp_model = EarlyFusionPROD_Resnet18()
efp_model.to(device)
save_path = "./saved_models/bs_16_lr_1e-05/EFP_F1_{}.pt"
file_path = "./saved_results/bs_16_lr_1e-05/EFP_results.pkl"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(efp_model.parameters(), lr=lr)
efp_results = []
acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict = train_model(efp_model, dataloader, optimizer, criterion, num_epochs=10, save_path=save_path)
efp_results.append((acc_dict, loss_dict, kappa_dict, f1_dict, auc_dict))
with open(file_path, 'wb') as file:
    pickle.dump(efp_results, file)

----------
Epoch: 1/10
----------
Phase:train, epoch loss: 0.2221
Phase:validate, epoch loss: 0.1774
Kappa: 0.7116 F1: 0.8558 AUC: 0.6443
Phase:test, epoch loss: 0.1768
Kappa: 0.7082 F1: 0.8541 AUC: 0.5890
Epoch complete in 12m 25s
----------
Epoch: 2/10
----------
Phase:train, epoch loss: 0.1641
Phase:validate, epoch loss: 0.1652
Kappa: 0.7119 F1: 0.8559 AUC: 0.6816
Phase:test, epoch loss: 0.1644
Kappa: 0.7094 F1: 0.8547 AUC: 0.6228
Epoch complete in 10m 15s
----------
Epoch: 3/10
----------
Phase:train, epoch loss: 0.1520
Phase:validate, epoch loss: 0.1612
Kappa: 0.7141 F1: 0.8571 AUC: 0.6992
Phase:test, epoch loss: 0.1609
Kappa: 0.7106 F1: 0.8553 AUC: 0.6436
Epoch complete in 9m 54s
----------
Epoch: 4/10
----------
Phase:train, epoch loss: 0.1440
Phase:validate, epoch loss: 0.1584
Kappa: 0.7184 F1: 0.8592 AUC: 0.7186
Phase:test, epoch loss: 0.1605
Kappa: 0.7089 F1: 0.8545 AUC: 0.6519
Epoch complete in 10m 14s
----------
Epoch: 5/10
----------
Phase:train, epoch loss: 0.1356
Phase:v